In [ ]:
data=df_matrix.resample("D").sum().copy()

In [ ]:
long=[e for e in data.columns if "long" in e]
short=[e for e in data.columns if "short" in e]

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import ward, fcluster
import random 

Z= data[long].loc[:"2022-11-01"].corr()  ################  bigdf_PTN_DD sul drawdown oppure bigdf_PTN sulle operazioni

Zlista = Z.index.values.tolist()

linked = linkage(Z, 'single')

labelList = Zlista

level = 50 ######### numero di cluster 

plt.figure(figsize=(15, 10))
dendro=dendrogram(linked,
            orientation='top',
            labels=labelList,
            distance_sort='Descending',
            truncate_mode='lastp', 
            show_leaf_counts=True,
            p=level)
plt.show()


label = fcluster(linked, level, criterion='maxclust')

df_clst = pd.DataFrame()
df_clst["index"]  = Z.index
df_clst['label']  = label

LS=[]
for i in range(level):
    elements = df_clst[df_clst['label']==i+1]['index'].tolist()  
    size = len(elements)
    print('\n Cluster {}: N = {}'.format(i+1, size))
    LS.append(elements)


ok=[]
ok2=[]
for idn in tqdm(range(len(LS))):
    v_max=[]
    x=data[LS[idn]]
    if len(x.columns)>0:
        for k in x.columns:
            g=data[k].loc[:"2022-11-01"]
            g=g.where(g!=0).dropna()
            
            p=profit_factor(g)
            k=kestner_ratio(g)
            c=calmar_ratio(g)
            sr=old_sharpe_ratio(g)
            m_var=-np.var(g)
            g=(p+k+c)#*len(g)#*g.sum()
            av=avg_trade(g)

            v_max.append(av)#(random.choice([p,k,c,sr,m_var,av])) ######## qui metti cosa vuoi valutare
           
        if len(v_max)>0:
            r=v_max.index(max(v_max))
            ok2.append(max(v_max))
            ok.append(x.iloc[:,r].name)

ls_d=data[ok]#df_matrix[ok]

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import ward, fcluster



Z= data[short].loc[:"2022-11-01"].corr()  ################  bigdf_PTN_DD sul drawdown oppure bigdf_PTN sulle operazioni

Zlista = Z.index.values.tolist()

linked = linkage(Z, 'single')

labelList = Zlista

level = 50 ######### numero di cluster 

plt.figure(figsize=(15, 10))
dendro=dendrogram(linked,
            orientation='top',
            labels=labelList,
            distance_sort='Descending',
            truncate_mode='lastp', 
            show_leaf_counts=True,
            p=level)
plt.show()


label = fcluster(linked, level, criterion='maxclust')

df_clst = pd.DataFrame()
df_clst["index"]  = Z.index
df_clst['label']  = label

LS=[]
for i in range(level):
    elements = df_clst[df_clst['label']==i+1]['index'].tolist()  
    size = len(elements)
    print('\n Cluster {}: N = {}'.format(i+1, size))
    LS.append(elements)


ok=[]
ok2=[]
for idn in tqdm(range(len(LS))):
    v_max=[]
    x=data[LS[idn]]
    if len(x.columns)>0:
        for k in x.columns:
            g=data[k].loc[:"2022-11-01"]
            g=g.where(g!=0).dropna()
            
            p=profit_factor(g)
            k=kestner_ratio(g)
            c=calmar_ratio(g)
            sr=old_sharpe_ratio(g)
            m_var=-np.var(g)
            g=(p+k+c)#*len(g)#*g.sum()
            av=avg_trade(g)

            v_max.append(av)#(random.choice([p,k,c,sr,m_var,av])) ######## qui metti cosa vuoi valutare
            #v_max.append(m_var)
        if len(v_max)>0:
            r=v_max.index(max(v_max))
            ok2.append(max(v_max))
            ok.append(x.iloc[:,r].name)

ls_s=data[ok]#df_matrix[ok]

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest , chi2,GenericUnivariateSelect
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer, accuracy_score

DF=df_matrix.resample("D").sum()

dataset=carica_storico_fast(SIMBOLO,"UTC",0,"60min","2010-01-01","2024-01-01",0,0)
dataset=dataset[["open","high","low","close","volume"]]
dataset = dataset.resample("D").agg({'open': 'first','high': 'max','low':'min','close':'last','volume':'sum'}).dropna()



In [ ]:
df_l=pd.DataFrame()
for e in tqdm(range(len(DF.columns))):
    a=DF.iloc[:,e].to_frame()
    #a=a.loc["2019-01-01":]
    a=a[a.index.dayofweek<5]
    a.columns=(["operations"])
    a["cumulato"]=a.operations.cumsum()
    a["cumulato_mean5"]=a["cumulato"].rolling(5).mean()
    a["cumulato_mean30"]=a["cumulato"].rolling(30).mean()
    a["cumulato_mean100"]=a["cumulato"].rolling(100).mean()
    a["cumulato_log"]=np.log(a["cumulato"])         
    a["cumulato_log"][a["cumulato_log"]==-np.inf]=0
    a["cumulato_max"]=a.operations.cummax()
    a["cumulato_min"]=a.operations.cummin()
    
    tmp_pf=a.operations[a.operations!=0].expanding().apply(profit_factor)
    a["profit_factor_exp"]=tmp_pf
    a["profit_factor_exp"]=a["profit_factor_exp"].fillna(method="ffill")
    a["profit_factor_exp"]=a["profit_factor_exp"].fillna(0)
    tmp_pf=a.operations[a.operations!=0].rolling(30).apply(profit_factor)
    a["profit_factor_rol"]=tmp_pf
    a["profit_factor_rol"]=a["profit_factor_rol"].fillna(method="ffill")
    a["profit_factor_rol"]=a["profit_factor_rol"].fillna(0)
    a["drawdown"]=drawdown(a.cumulato)
    a["drawdown_inv"]=a["drawdown"]*(-1)
    
    a["price_close"]=dataset.close
    
    a["price_close_50"]=dataset.close.rolling(50).mean()
    a["price_close_30"]=dataset.close.rolling(30).mean()
    a["price_close_50_inv"]=a["price_close_50"]*(-1)
    a["price_close_30_inv"]=a["price_close_30"]*(-1) 
    a["price_close_80"]=dataset.close.rolling(80).mean()
    a["price_close_100"]=dataset.close.rolling(100).mean()
    a["price_close_80_inv"]=a["price_close_80"]*(-1)
    a["price_close_100_inv"]=a["price_close_100"]*(-1)
    
    a["price_close_inv"]=dataset.close*(-1)
    a["price_close_pct_1"]=a.price_close.pct_change(1)
    a["price_close_pct_1"]=a["price_close_pct_1"].fillna(0)
    a["price_close_pct_inv_1"]=a["price_close_pct_1"]*(-1)
    
    #a["price_close_pass"]=dataset.close.shift(1)
    
    a["price_volume"]=dataset.volume
    a["price_volume_pct"]=a.price_volume.pct_change(1)
    a["price_volume_pct"]=a["price_volume_pct"].fillna(0)
    a["price_volume_pct_inv"]=a["price_volume_pct"]*(-1)
    ########################################################

    X=a.iloc[:,1:].copy()
    X=X.fillna(0)
    y=a.iloc[:,0].copy()
    y[y>0]=1
    y[y<0]=2
    y=y.shift(-1)
    y=y.fillna(0)

    #######################################################

    

    #######################################################

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(y),
                                            y = y                                                    
                                        )
    class_weights = dict(zip(np.unique(y), class_weights))
    
    selector = SelectKBest(f_regression,k=3) #10
    selector.fit_transform(X, y) 
    columns_idx = selector.get_support(indices=True)
    X=X.iloc[:,columns_idx]
    print(X.columns)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15 , random_state=42,shuffle=False)
    #######################################################

    #model = HistGradientBoostingClassifier(loss= 'auto' ,random_state=0,max_iter=10 ,class_weight=class_weights, max_depth=10) #max_iter=3 #max_depth=None
    model = RandomForestClassifier(n_estimators=11, criterion='gini', max_depth=7, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=-1, random_state=1, verbose=0, warm_start=False, class_weight=class_weights, ccp_alpha=0.0, max_samples=None)

    
    model.fit(X_train, y_train)
    predict=model.predict(X)
    predict=np.where(predict==2,-1,predict)

    #######################################################
    
    

    a["predict"]=predict
    ac=accuracy_score(y_train , a["predict"].loc[:y_train.index[-1]])
    print("Accuracy:",round(ac,2))
    
    if (ac>0.6)&(ac<0.95):
        a["controlled_operations"]=a.operations*a.predict.shift(1)
        a[["controlled_operations","operations"]].cumsum().plot(title=DF.iloc[:,e].name)
        plt.axvline(x=y_test.index[0],color="red")

        plt.show()

        b=a["controlled_operations"].to_frame()
        b.columns=([DF.iloc[:,e].name])
        df_l=pd.concat([df_l,b],axis=1).fillna(0)

In [ ]:
df_l.sum(axis=1).loc[y_test.index[0]:].cumsum().plot(figsize=(10,10),label="EQC")
plt.legend(loc='upper left')

In [ ]:
df_l.sum(axis=1).loc[y_test.index[0]:].cumsum().plot(figsize=(10,10),label="EQC")
plt.legend(loc='upper left')
DF.sum(axis=1).loc[y_test.index[0]:].cumsum().plot(secondary_y=True,label="ORIGINALE")
plt.legend(loc='upper left')
plt.show()